<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2021**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai<br/>
**Names**: Kimon Vogt and Ana Vitoria Rodrigues Lima<br/>


<hr style='height:2px'>

In [63]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain questions about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using [Jupyter Notebooks](https://jupyter.org/), which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with [pandas](https://pandas.pydata.org/) as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- <span style='color:purple'>**Web scraping requests can take several minutes**</span>. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, <span style='color:purple'>the order in which you run cells matters</span> and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All"). Just make sure to not re-run the time intensive tasks unnecessarily. In this notebook for example, you could declare a variable to act as a 'setting' and use some controll logic to prevent a re-scrap from happening when not desired.

- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data with [Requests](https://docs.python-requests.org/en/master/) and then manipulating it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). 

But first, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [65]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import pickle # for loading a dictionary from disk
from typing import Optional # typehint that value can also be None

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# data/
# state_data/
#state_dir = "AC209a/HW1/state_data/"
state_dir = 'state_data/'

In [66]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [67]:
# YOUR CODE HERE

home_page = requests.get(base_url)

# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- **When working with Jupyter Notebooks, avoiding unnecessarily long output in is essential.** Write code that prints the first 10,000 characters from the contents of `home_page` and [enable scolling output for the cell](https://www.youtube.com/watch?v=U4usAUZCv_c&t=1s).</div>


In [68]:
# YOUR CODE HERE

print(home_page)
raw_home=home_page.text
print(home_page.text[:10000])

# END OF YOUR CODE HERE

<Response [200]>
<!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"1475026924","applicationTime":1880.271443,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"e640ef2bbcc3dd09","agentToken":null}; (window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"1475026924"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n){return function(){return o(t,[u.now()].concat(f(arguments)),e?null:this,n),e?void 0:this}}var o=t("handle"),a=t

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [69]:
# YOUR CODE HERE

soup = BeautifulSoup(raw_home, 'html.parser')
soup

pretty_soup= soup.prettify()
pretty_soup


# END OF YOUR CODE HERE

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <script type="text/javascript">\n   window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"1475026924","applicationTime":1880.271443,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"e640ef2bbcc3dd09","agentToken":null}; (window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"1475026924"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n){return function(){return o(t,[u.now()].concat(f(arguments)),e?null:this,n),e?void 0:this}}var o=t("handle"),a=t(8

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use [Regular Expressions]('https://docs.python.org/3/library/re.html'), in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate a `state_urls` [dictionary]('https://docs.python.org/3/tutorial/datastructures.html#dictionaries') by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later exercises go smoothly.
- Some HTML tag attributes may change over time. It your code stops working, make sure you are not targeting such ephemeral elements ('jss' class attributes are a common culprit)
</div>

In [70]:
import urllib 

# YOUR CODE HERE
state_urls = {}
pretty_soup = soup.prettify()

list_a_tag = soup.select("th a")
list_a_tag

for i in list_a_tag:
    state_urls[i.text]=urllib.parse.urljoin(base_url, i.get('href'))

state_urls
# END OF YOUR CODE HERE

{'Alabama': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/alabama',
 'Alaska': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/alaska',
 'Arizona': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/arizona',
 'Arkansas': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/arkansas',
 'California': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/california',
 'Colorado': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/colorado',
 'Connecticut': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/connecticut',
 'Delaware': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/delaware',
 'District of Columbia': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia',
 'Florida': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/st

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [72]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir`, defined above, as the prefix to the path.) 

**NOTES:**
- **Leave a 2 second pause between requests**
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
- **Once you have written the files you can comment out this cell. This will save time and prevent you from making unnecessary requests when you restart the kernel & re-run all cells in the noteboook before submitting (as you should!)**
</div>

In [74]:
# 1.5 (5 pts) -- save each webpage to disk
#state_dir = "/state_data/"

import os as os
import sys

for state, url in state_urls.items():
    # YOUR CODE HERE
    name_of_file = state
    completeName= os.path.join(state_dir, name_of_file)
    file=open(completeName,'w')
    toFile = requests.get(url).text
    
    file.write(toFile)
    file.close()

    # END OF YOUR CODE HERE
    
    sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that [pandas](https://pandas.pydata.org/) is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without pandas. Thus, here we ask you to perform a few operations without using pandas, and then in Exercise 3 we will use pandas.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`
This is an example of a Python [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [75]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]
print(state_info)

[('Alabama', 'state_data/Alabama'), ('Alaska', 'state_data/Alaska'), ('Arizona', 'state_data/Arizona'), ('Arkansas', 'state_data/Arkansas'), ('California', 'state_data/California'), ('Colorado', 'state_data/Colorado'), ('Connecticut', 'state_data/Connecticut'), ('Delaware', 'state_data/Delaware'), ('District of Columbia', 'state_data/District of Columbia'), ('Florida', 'state_data/Florida'), ('Georgia', 'state_data/Georgia'), ('Hawaii', 'state_data/Hawaii'), ('Idaho', 'state_data/Idaho'), ('Illinois', 'state_data/Illinois'), ('Indiana', 'state_data/Indiana'), ('Iowa', 'state_data/Iowa'), ('Kansas', 'state_data/Kansas'), ('Kentucky', 'state_data/Kentucky'), ('Louisiana', 'state_data/Louisiana'), ('Maine', 'state_data/Maine'), ('Maryland', 'state_data/Maryland'), ('Massachusetts', 'state_data/Massachusetts'), ('Michigan', 'state_data/Michigan'), ('Minnesota', 'state_data/Minnesota'), ('Mississippi', 'state_data/Mississippi'), ('Missouri', 'state_data/Missouri'), ('Montana', 'state_data/M

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of [tuples](https://docs.python.org/3.3/library/stdtypes.html?highlight=tuple#tuple): (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - 7 day average case
    - 7 day average deaths
    - \# of confirmed cases (total)
    - \# of deaths
    - Stores the above 4 pieces of data above as well as **population** in a **non-pandas** data structure named `covid_data` **for every county across every state**
- Returns `covid_data`
    <font color='blue'>


**NOTES:**
- **Attention: the population variable not in `state_info`. More on info on where to get this value is found in the green block below**
- To be clear, as of September 7, 2021, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama.
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
- For the duration of our using this data for the homework, be sure to **properly store the data with the correct data types;** that is, counts should be represented as Integers and rates should be represented as Floats. For example:
    - \# of confirmed cases (total) should be an **Integer**
    - \# of deaths should be an **Integer**
    - \# of confirmed cases (per 100k) should be a **Float** (we haven't created this feature yet!)
    - 7 day average cases should be an **Integer** (you'd think an average should be a float but the values you scrapped were rounded to the nearest int)
</div>

<div style='background-color:lightgreen;padding:15px'>
    <strong>Injecting population data</strong>
    

The table on usafacts.org you've just scrapped originally had additional columns related to county population. But these have recently been removed! We'd like you to be able to utilize the population data in the following section but also use up-to-date COVID data (so the [Internet Archive](https://archive.org/) was not an option). And, though this information is available elsewhere on usafacts.org, we've decided that you've already done enough web scraping for one HW. So below we've provided a [kludge](https://en.wikipedia.org/wiki/Kludge#Computer_science).
    
`population_dict` is a nested dictionary. The keys are states whose values are _themselves_ dictionaries. Those '_inner_' dictionaries' keys are counties and their values are populations. It looks like this:
```python
{'Alabama': {'Autauga County': 55869,
             'Baldwin County': 223234,
           ...
'Wyoming': {'Albany County': 38880,
            'Big Horn County': 11790,
            ...
```

To get at a population you could use double dictionary indexing like `population_dict['Alabama']['Autauga County']`

But not all of the counties you've scrapped have population data in this dictionary. So we've provided a helper function, `get_pop`, that will return `None` if the county data was not found. Use `get_pop` to inject popoulation data into your `covid_data` as you build it up in the `load_covid_data` function you'll implement below.
    
**Final Note: you should _ignore counties with missing population data or populations of 0_. Simply do not add them to `covid_data` as it is constructed.**
</div>

In [76]:
# load additional county population data as a nested dictionary
# you can read about this strange .pkl 'pickle' file here
# https://docs.python.org/3/library/pickle.html
with open('population.pkl', 'rb') as f:
    population_dict = pickle.load(f)
    

# not sure what's happening with the data types in the function header?
# check out: https://docs.python.org/3/library/typing.html#module-typing
def get_pop(state: str, county: str) -> Optional[int]:
    '''
    returns population of country, state (int)
    If county or state not found, returns None
    Example: get_pop('Alabama', 'Autauga County')
    '''
    try:
        return population_dict.get(state).get(county)
    except AttributeError:
        print('incorrect state name!')
        return None

In [77]:
def load_covid_data(state_info):
    covid_data = {}
    # YOUR CODE HERE
    for i in range(len(state_info)):
        state_website_content = BeautifulSoup(open(state_info[i][1],'r').read(), 'html.parser')
        table_node = state_website_content.select('table')[1]
        th_list = table_node.select('th a')
        td_list = table_node.select('td')
        print('State:', state_info[i][0])
        print('State file:', state_info[i][1])
        print("Number of headers in table:", len(table_node.select('th a')))
        print("Number of data in table:", len(table_node.select('td')))
        
        covid_data[state_info[i][0]] = {}
        for j, th in enumerate(th_list):
            if get_pop(state_info[i][0], th.text):
                covid_data[state_info[i][0]][th.text]={}
                #print(th.text)
                #do a loop with all the data related to that county
                #for k in range(4):
                covid_data[state_info[i][0]][th.text]['seven_d_cases']= int(td_list[4*j ].text.replace(',', ''))
                covid_data[state_info[i][0]][th.text]['seven_d_deaths']= int(td_list[4*j + 1].text.replace(',', ''))
                covid_data[state_info[i][0]][th.text]['cases']= int(td_list[4*j + 2].text.replace(',', ''))
                covid_data[state_info[i][0]][th.text]['deaths']= int(td_list[4*j + 3].text.replace(',', ''))
        
    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [78]:
covid_data = load_covid_data(state_info)

State: Alabama
State file: state_data/Alabama
Number of headers in table: 67
Number of data in table: 268
State: Alaska
State file: state_data/Alaska
Number of headers in table: 29
Number of data in table: 116
State: Arizona
State file: state_data/Arizona
Number of headers in table: 15
Number of data in table: 60
State: Arkansas
State file: state_data/Arkansas
Number of headers in table: 75
Number of data in table: 300
State: California
State file: state_data/California
Number of headers in table: 58
Number of data in table: 232
State: Colorado
State file: state_data/Colorado
Number of headers in table: 64
Number of data in table: 256
State: Connecticut
State file: state_data/Connecticut
Number of headers in table: 8
Number of data in table: 32
State: Delaware
State file: state_data/Delaware
Number of headers in table: 3
Number of data in table: 12
State: District of Columbia
State file: state_data/District of Columbia
Number of headers in table: 1
Number of data in table: 4
State: Flo

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people
   
**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any one of the tied counties (see if you can do it in an unbiased way!)
</div>

In [79]:
def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    county_index=[]
    county_value_oneperst=[]
    for i in covid_data:
        #print( i)
        temp= []
        for j in covid_data[i]:
            temp.append(covid_data[i][j]['cases']/population_dict[i][j]*(100000))
        max_val= max(temp)
        county_value_oneperst.append(max_val)
        county_index.append(temp.index(max_val))
    
    max_overall_value=max(county_value_oneperst)
    index_want=county_value_oneperst.index(max_overall_value)
    state = list(covid_data.keys())[index_want]
    county = list(covid_data[state].keys())[county_index[index_want]]
    print('County that has the HIGHEST rate of COVID cases per 100k people:', county_index[index_want], county)
    print('State of the aforementioned county:', state_info[index_want][0])
    print('Rate of COVID cases per 100k people in aforementioned county:', max_overall_value)
    
    min_county_index=[]
    min_county_value_oneperst=[]
    for i in covid_data:
        temp= []
        for j in covid_data[i]:
            temp.append(covid_data[i][j]['cases']/population_dict[i][j]*(100000))
        min_val= min(temp)
        min_county_value_oneperst.append(min_val)
        min_county_index.append(temp.index(min_val))
    
    min_overall_value=min(min_county_value_oneperst)
    min_index_want=min_county_value_oneperst.index(min_overall_value)
    min_state = list(covid_data.keys())[min_index_want]
    min_county = list(covid_data[min_state].keys())[min_county_index[min_index_want]]
    print('')
    print('')
    print('County that has the LOWEST rate of COVID cases per 100k people:',min_county_index[min_index_want], min_county)
    print('State of the aforementioned county:',state_info[min_index_want][0])
    print('Rate of COVID cases per 100k people in aforementioned county:', min_overall_value)
    
    return      
    # END OF YOUR CODE HERE
    

Run the cell below (no changes necessary) to execute your code above

In [80]:
calculate_county_stats(covid_data)

County that has the HIGHEST rate of COVID cases per 100k people: 3 Bristol Bay Borough
State of the aforementioned county: Alaska
Rate of COVID cases per 100k people in aforementioned county: 72727.27272727274


County that has the LOWEST rate of COVID cases per 100k people: 12 Lake and Peninsula Borough
State of the aforementioned county: Alaska
Rate of COVID cases per 100k people in aforementioned county: 0.0


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied states

</div>

In [81]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
    county_index=[]
    county_value_oneperst=[]
    lista=[]
    for i in covid_data:
        temp=0
        for j in covid_data[i]:
            temp+=covid_data[i][j]['deaths']
        lista.append(temp)
#    print(len(lista))
            
    ab=max(lista)
    ac=min(lista)
    high=lista.index(ab)
    low=lista.index(ac)
    q=list(covid_data.keys())[high]
    q2=list(covid_data.keys())[low]
    print('This is the state that has the HIGHEST number of deaths:',q)
    print('')
    print('This is the state that has the LOWEST number of deaths:',q2)     
    
    return      
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [82]:
calculate_state_deaths(covid_data)

This is the state that has the HIGHEST number of deaths: California

This is the state that has the LOWEST number of deaths: Hawaii


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [83]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    county_index=[]
    county_value_oneperst=[]
    lista=[]

    for i in covid_data:
        temp=0
        temp2=0
        for j in covid_data[i]:
            temp+=covid_data[i][j]['deaths']
            temp2+=population_dict[i][j]
        a = temp/temp2
        lista.append(a)
    #print(len(lista))
    ab=max(lista)
    ac=min(lista)
    high=lista.index(ab)
    low=lista.index(ac)
    #print(high,low)
    q=list(covid_data.keys())[high]
    q2=list(covid_data.keys())[low]
    print('This is the state that has the HIGHEST number of deaths based on its entire population:',q)
    print('')
    print('This is the state that has the LOWEST number of deaths based on its entire population:',q2)     #max_overall_value=max(county_value_oneperst)
    
    return      
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [84]:
calculate_state_deathrate(covid_data)

This is the state that has the HIGHEST number of deaths based on its entire population: Mississippi

This is the state that has the LOWEST number of deaths based on its entire population: Hawaii


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 4 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# case per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 2 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [85]:
new_df = pd.DataFrame.from_dict({(i,j): covid_data[i][j]
                                 for i in covid_data.keys()
                                 for j in covid_data[i].keys()},
                                orient = 'index')
    
covid_df = new_df.reset_index(inplace = False)
covid_df.rename(columns = {'level_0':'state',
                           'level_1':'county'}, inplace = True)
covid_df

,state,county,seven_d_cases,seven_d_deaths,cases,deaths
0,Alabama,Autauga County,41,0,9351,119
1,Alabama,Baldwin County,139,4,35221,417
2,Alabama,Barbour County,17,0,3329,67
3,Alabama,Bibb County,28,0,3859,75
4,Alabama,Blount County,42,0,9246,146
...,...,...,...,...,...,...
3076,Wyoming,Sweetwater County,69,0,6275,51
3077,Wyoming,Teton County,25,0,4679,11
3078,Wyoming,Uinta County,28,0,3248,16
3079,Wyoming,Washakie County,10,0,1148,27


In [28]:
population_dict

{'Alabama': {'Autauga County': 55869,
  'Baldwin County': 223234,
  'Barbour County': 24686,
  'Bibb County': 22394,
  'Blount County': 57826,
  'Bullock County': 10101,
  'Butler County': 19448,
  'Calhoun County': 113605,
  'Chambers County': 33254,
  'Cherokee County': 26196,
  'Chilton County': 44428,
  'Choctaw County': 12589,
  'Clarke County': 23622,
  'Clay County': 13235,
  'Cleburne County': 14910,
  'Coffee County': 52342,
  'Colbert County': 55241,
  'Conecuh County': 12067,
  'Coosa County': 10663,
  'Covington County': 37049,
  'Crenshaw County': 13772,
  'Cullman County': 83768,
  'Dale County': 49172,
  'Dallas County': 37196,
  'DeKalb County': 71513,
  'Elmore County': 81209,
  'Escambia County': 36633,
  'Etowah County': 102268,
  'Fayette County': 16302,
  'Franklin County': 31362,
  'Geneva County': 26271,
  'Greene County': 8111,
  'Hale County': 14651,
  'Henry County': 17205,
  'Houston County': 105882,
  'Jackson County': 51626,
  'Jefferson County': 658573,
  

In [86]:
pop_county_list = []
for state in population_dict.keys():
    state_county_pop_i = pd.DataFrame.from_dict(population_dict[state], orient = "index")
    state_county_pop_i['state'] = state
    state_county_pop_i = state_county_pop_i.reset_index(inplace = False)
    state_county_pop_i.columns = ['county', 'population', 'state']
    state_county_pop_i = state_county_pop_i[['state', 'county', 'population']]
    pop_county_list.append(state_county_pop_i)
# print(pop_county_list)
state_county_pop = pd.concat(pop_county_list)
state_county_pop



,state,county,population
0,Alabama,Autauga County,55869
1,Alabama,Baldwin County,223234
2,Alabama,Barbour County,24686
3,Alabama,Bibb County,22394
4,Alabama,Blount County,57826
...,...,...,...
18,Wyoming,Sweetwater County,42343
19,Wyoming,Teton County,23464
20,Wyoming,Uinta County,20226
21,Wyoming,Washakie County,7805


In [87]:
covid_df

covid_df_merged = pd.merge(covid_df, state_county_pop,
                           how = 'left',
                           left_on = ['state','county'], right_on = ['state','county'])

covid_df_merged['cases_per_100k'] = covid_df_merged['cases'] / covid_df_merged['population'] * 100000
covid_df_merged['cases_per_100k'] = covid_df_merged['cases_per_100k'].astype('int')
covid_df_merged

,state,county,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
0,Alabama,Autauga County,41,0,9351,119,55869,16737
1,Alabama,Baldwin County,139,4,35221,417,223234,15777
2,Alabama,Barbour County,17,0,3329,67,24686,13485
3,Alabama,Bibb County,28,0,3859,75,22394,17232
4,Alabama,Blount County,42,0,9246,146,57826,15989
...,...,...,...,...,...,...,...,...
3076,Wyoming,Sweetwater County,69,0,6275,51,42343,14819
3077,Wyoming,Teton County,25,0,4679,11,23464,19941
3078,Wyoming,Uinta County,28,0,3248,16,20226,16058
3079,Wyoming,Washakie County,10,0,1148,27,7805,14708


In [88]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    # first, make a dataframe from dictionary values of covid data
    new_df = pd.DataFrame.from_dict({(i,j): covid_data[i][j] 
                           for i in covid_data.keys() 
                           for j in covid_data[i].keys()},
                       orient = 'index')
    
    covid_df = new_df.reset_index(inplace = False)
    covid_df.rename(columns = {'level_0':'state',
                               'level_1':'county'}, inplace = True)
    
    # second, make another dataframe
    # from dictionary values of population data
    pop_county_list = []
    for state in population_dict.keys():
        state_county_pop_i = pd.DataFrame.from_dict(population_dict[state], orient = "index")
        state_county_pop_i['state'] = state
        state_county_pop_i = state_county_pop_i.reset_index(inplace = False)
        state_county_pop_i.columns = ['county', 'population', 'state']
        state_county_pop_i = state_county_pop_i[['state', 'county', 'population']]
        pop_county_list.append(state_county_pop_i)
    state_county_pop = pd.concat(pop_county_list)
    
    # third, merge the two dataframes so that we can compute cases per 100k
    covid_df_merged = pd.merge(covid_df, state_county_pop,
                           how = 'left',
                           left_on = ['state','county'], right_on = ['state','county'])
    covid_df_merged['cases_per_100k'] = covid_df_merged['cases'] / covid_df_merged['population'] * 100000
    
    # covid_df_merged['cases_per_100k'] = covid_df_merged['cases_per_100k'].astype('int')

    
    # END OF YOUR CODE HERE
    return covid_df_merged

Run the cell below (no changes necessary) to execute your code above and inspect the results.

In [89]:
covid_df = convert_to_pandas(covid_data)
covid_df


,state,county,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
0,Alabama,Autauga County,41,0,9351,119,55869,16737.367771
1,Alabama,Baldwin County,139,4,35221,417,223234,15777.614521
2,Alabama,Barbour County,17,0,3329,67,24686,13485.376327
3,Alabama,Bibb County,28,0,3859,75,22394,17232.294365
4,Alabama,Blount County,42,0,9246,146,57826,15989.347352
...,...,...,...,...,...,...,...,...
3076,Wyoming,Sweetwater County,69,0,6275,51,42343,14819.450677
3077,Wyoming,Teton County,25,0,4679,11,23464,19941.186498
3078,Wyoming,Uinta County,28,0,3248,16,20226,16058.538515
3079,Wyoming,Washakie County,10,0,1148,27,7805,14708.520179


<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information (other than ties) as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [90]:
# max(covid_df['cases_per_100k']) == covid_df['cases_per_100k']
maxrow_cases_per_100k = covid_df['cases_per_100k'].idxmax()
# covid_df.iloc[maxrow_cases_per_100k]

minrow_cases_per_100k = covid_df['cases_per_100k'].idxmin()
# covid_df.iloc[minrow_cases_per_100k]

covid_df.iloc[[minrow_cases_per_100k, maxrow_cases_per_100k]]


,state,county,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
79,Alaska,Lake and Peninsula Borough,0,0,0,0,1592,0.000000
70,Alaska,Bristol Bay Borough,0,0,608,2,836,72727.272727


In [91]:
def calculate_county_stats2(df):

    # YOUR CODE HERE
    maxrow_cases_per_100k = covid_df['cases_per_100k'].idxmax()
    minrow_cases_per_100k = covid_df['cases_per_100k'].idxmin()
    
    df_maxmin = df.iloc[[minrow_cases_per_100k, maxrow_cases_per_100k]]
    
    return df_maxmin
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [92]:
calculate_county_stats2(covid_df)

,state,county,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
79,Alaska,Lake and Peninsula Borough,0,0,0,0,1592,0.000000
70,Alaska,Bristol Bay Borough,0,0,608,2,836,72727.272727


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3 (other than ties), but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied states
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [93]:
def calculate_state_deaths2(df):

    # YOUR CODE HERE
    # first need to group and sum counties into states
    df_bystate = df.groupby(['state']).sum()

    # and add a state column
    df_bystate = df_bystate.reset_index(inplace = False)


    # but need to recalculate the cases_per_100k, since it doesn't make sense to sum a rate
    df_bystate['cases_per_100k'] = df_bystate['cases'] / df_bystate['population'] * 100000

    # max(covid_df['cases_per_100k']) == covid_df['cases_per_100k']
    maxrow_deaths = df_bystate['deaths'].idxmax()
    minrow_deaths = df_bystate['deaths'].idxmin()

    df_maxmin = df_bystate.iloc[[minrow_deaths, maxrow_deaths]]
    
    return df_maxmin
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [94]:
calculate_state_deaths2(covid_df)

,state,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
11,Hawaii,177,0,19510,152,441309,4420.938617
4,California,7417,83,4321742,66016,38630674,11187.332636


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.** That is, return:

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [95]:
def calculate_state_deathrate2(df):

    # YOUR CODE HERE
    # first need to group and sum counties into states
    df_bystate = df.groupby(['state']).sum()

    # and add a state column
    df_bystate = df_bystate.reset_index(inplace = False)


    # but need to recalculate the cases_per_100k, since it doesn't make sense to sum a rate
    df_bystate['cases_per_100k'] = df_bystate['cases'] / df_bystate['population'] * 100000
    
    # calculate deathrate
    df_bystate['deathrate'] = df_bystate['deaths'] / df_bystate['population']

    # max(covid_df['cases_per_100k']) == covid_df['cases_per_100k']
    maxrow_deathrate = df_bystate['deathrate'].idxmax()
    minrow_deathrate = df_bystate['deathrate'].idxmin()

    df_maxmin = df_bystate.iloc[[minrow_deathrate, maxrow_deathrate]]
    
    return df_maxmin
    # END OF YOUR CODE HERE

print(calculate_state_deathrate2(covid_df)['state'].iloc[0] + " has the lowest COVID death rate; 1 out of every " + str(int(np.reciprocal(calculate_state_deathrate2(covid_df)['deathrate'].iloc[0]))) + " people has died")
print(calculate_state_deathrate2(covid_df)['state'].iloc[1] + " has the highest COVID death rate; 1 out of every " + str(int(np.reciprocal(calculate_state_deathrate2(covid_df)['deathrate'].iloc[1]))) + " people has died")

Hawaii has the lowest COVID death rate; 1 out of every 2903 people has died
Mississippi has the highest COVID death rate; 1 out of every 328 people has died


Run the cell below (no changes necessary) to execute your code above

In [96]:
calculate_state_deathrate2(covid_df)

,state,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k,deathrate
11,Hawaii,177,0,19510,152,441309,4420.938617,0.000344
24,Mississippi,1827,47,466145,9061,2976149,15662.690275,0.003045


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 3 new columns:
- `case_fatality_rate`
- `# covid deaths per 100k` and
- `population`

And return the updated DataFrame **sorted by `case_fatality_rate` in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
- Make sure you inspect your results thoroughly. You may have to address the results of divisions by zero (or prevent these divisions in the first place). 
</div>

In [97]:
covid_df

,state,county,seven_d_cases,seven_d_deaths,cases,deaths,population,cases_per_100k
0,Alabama,Autauga County,41,0,9351,119,55869,16737.367771
1,Alabama,Baldwin County,139,4,35221,417,223234,15777.614521
2,Alabama,Barbour County,17,0,3329,67,24686,13485.376327
3,Alabama,Bibb County,28,0,3859,75,22394,17232.294365
4,Alabama,Blount County,42,0,9246,146,57826,15989.347352
...,...,...,...,...,...,...,...,...
3076,Wyoming,Sweetwater County,69,0,6275,51,42343,14819.450677
3077,Wyoming,Teton County,25,0,4679,11,23464,19941.186498
3078,Wyoming,Uinta County,28,0,3248,16,20226,16058.538515
3079,Wyoming,Washakie County,10,0,1148,27,7805,14708.520179


In [98]:
def add_death_stats(df):
    
    # can add an infintesimal or fillna after the fact to handle nans from divide by 0.
    
    # YOUR CODE HERE
    df['deaths_per_100k'] = df['deaths'] / df['population'] * 100000
    df['case_fatality_rate'] = df['deaths'] / df['cases']
    
    # now sort by case fatality rate, ascending
    df = df.sort_values(by = 'case_fatality_rate', ascending = True)
    
    # fill NaNs
    df = df.fillna(0)
    
    # since we did fillna AFTER sorting, there are values of 0 at the head and at the tail
    # those at the head refer to counties with positive cases but zero deaths
    # those at the tail refer to counties with zero cases
    
    # select appropriate columns
    df = df[['state', 'county', 'population', 'cases', 'cases_per_100k', 'deaths', 'deaths_per_100k', 'case_fatality_rate']]
    
    # END OF YOUR CODE HERE
    return df

In [99]:
add_death_stats(covid_df)

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate
1675,Nebraska,Hayes County,922,61,6616.052061,0,0.000000,0.000000
1674,Nebraska,Harlan County,3380,219,6479.289941,0,0.000000,0.000000
2782,Utah,Wayne County,2711,186,6860.936924,0,0.000000,0.000000
2919,Washington,San Juan County,17582,277,1575.474918,0,0.000000,0.000000
1684,Nebraska,Keya Paha County,806,54,6699.751861,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1670,Nebraska,Grant County,623,41,6581.059390,4,642.054575,0.097561
434,Georgia,Glascock County,2971,176,5923.931336,19,639.515315,0.107955
1784,New Mexico,Harding County,625,14,2240.000000,2,320.000000,0.142857
79,Alaska,Lake and Peninsula Borough,1592,0,0.000000,0,0.000000,0.000000


Run the cell below (no changes necessary) to execute your code above

In [100]:
covid_updated = add_death_stats(covid_df)
covid_updated

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate
1675,Nebraska,Hayes County,922,61,6616.052061,0,0.000000,0.000000
1674,Nebraska,Harlan County,3380,219,6479.289941,0,0.000000,0.000000
2782,Utah,Wayne County,2711,186,6860.936924,0,0.000000,0.000000
2919,Washington,San Juan County,17582,277,1575.474918,0,0.000000,0.000000
1684,Nebraska,Keya Paha County,806,54,6699.751861,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1670,Nebraska,Grant County,623,41,6581.059390,4,642.054575,0.097561
434,Georgia,Glascock County,2971,176,5923.931336,19,639.515315,0.107955
1784,New Mexico,Harding County,625,14,2240.000000,2,320.000000,0.142857
79,Alaska,Lake and Peninsula Borough,1592,0,0.000000,0,0.000000,0.000000


In [101]:
covid_updated_sorted_fatality = covid_updated.sort_values(by='case_fatality_rate', axis=0, ascending=False)
covid_updated_sorted_fatality

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate
1784,New Mexico,Harding County,625,14,2240.000000,2,320.000000,0.142857
434,Georgia,Glascock County,2971,176,5923.931336,19,639.515315,0.107955
1670,Nebraska,Grant County,623,41,6581.059390,4,642.054575,0.097561
528,Georgia,Wilcox County,8635,610,7064.273306,50,579.038796,0.081967
417,Georgia,Dodge County,20605,1428,6930.356710,114,553.263771,0.079832
...,...,...,...,...,...,...,...,...
257,Colorado,Hinsdale County,820,52,6341.463415,0,0.000000,0.000000
1637,Nebraska,Blaine County,465,21,4516.129032,0,0.000000,0.000000
1311,Minnesota,Cook County,5463,206,3770.821893,0,0.000000,0.000000
219,California,Sierra County,3005,123,4093.178037,0,0.000000,0.000000


<div class='exercise'><b>Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6.1 [2 pts] Trends</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

<div style='background-color:#F6FEFA;padding:15px'>
 
In 3.3, the state with lowest number of deaths is Hawaii with 152 deaths, 19 510 cases and 4 420 cases per 100k people. The population of Hawaii is 441 309.
Whereas, in 3.3 the state with the highest number of deaths is California, with 66 016 deaths, 4 321 742 cases and 11 187 cases per 100k people. The population of California is 38 630 674.
    
In 3.4 the state with the lowest COVID death rate is Hawai with a rate of 1 out of 2903 people that died. The highest was Mississipi, with 1 out of 328 people. 
    
In 3.5 we computed the case fatality rate per state, and we can see that the highest fatality rate is in New Mexico with 0.14 - however the deaths were just 2.   
The state with the lowest case fatality is 0, Hawaii - which is in line with the two aforementioned results. 
    
Overall the trend noticed is that the higher the cases and the higher cases per 100k people are,  the higher the deaths and deaths per 100k people. However, it is not necessarily the case that the higher the case fatality rate is, the higher also the other variables (deaths, cases, deaths per 100k or cases per 100k). See the example of New Mexico mentioned above. We believe that country fatility rates and deaths is greatly ffected by the specifics of each country and the culture. 
    
    
    
</div>


<div class='exercise'><b>Exercise 3.6.2 [2 pts]: Data Reliability</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

<div style='background-color:#F6FEFA;padding:15px'>

In 3.5 we computed the case fatality rate. Although it is the highest in New Mexico with 0.1328, this piece of information alone is not a reliable point of reference to understand the gravity of the effects of the pandemic in a given state. From 3.3 and 3.4 we can see that California (with 66 016 deaths) and Mississipi (with 9 061 deaths)  have been impacted by the pandemic significantly more than New Mexico (with 2 deaths). 
As a consequence, the case fatality rate can not be observed on its own. Consider the difference in domentionality between the population_dict and our main data, we would agrue that the source is not reliable.
    

</div>


<div class='exercise'><b>Exercise 3.6.3 [1 pt]: Relationships Between Variables</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

<div style='background-color:#F6FEFA;padding:15px'>

To estimate cases from deaths, you would need some notion of case fatality rate. We have this figure in a column in the data, by county. But for a single county, we could use the fatality rate of nearby counties (say, the average case fatality rate for counties in the same state), and use that ratio to estimate the total number of cases. We wouldn't want to use the national rate because it can vary greatly over space, as we have shown in the exercise.


</div>


<div class='exercise'><b>Exercise 3.6.4 [1 pt]: Further Questions</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

<div style='background-color:#F6FEFA;padding:15px'>

It would be interesting to look at data related to other factors in the pandemic, such as:
    
    - correlation of masks sales and number of cases per 100k people in each state. We expect that the higher the masks sales and usage, the less the cases. It would be interesting to look at this.
    
    - correlation of average salary and number of cases. The higher the average salary of a state, the more likely it is that the main industry of that state is finance or tech, where there is the possility to work from home - hence less contagion opportunities. It would be interesting to see if this expecation is met by looking at salaries alongside with deaths and cases.
    
    - It would be interesting to look at the number of people using public transports in a given state, and see if there is a correlation between cases and deaths where the usage of public transport is more common. E.g. We assume that the usage of underground is more common in the state of NY rather than in the state of Texas. It would be interesting to look if these are correlated and if this expectation is met.
    
    
</div>


## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2020_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2020_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 8 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- \# covid deaths per 100k
- case_fatality_rate

along with these 15 columns from `election2020_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- voter_turnout
- voter_gap
- trump
- biden

**NOTES:**
- We are dropping two columns from `election2020_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful

**EXTRA INFORMATION:** In case you're wondering what the different features/columns are in `election2020_by_county.csv`:

- state: the state in which the county lies
- fipscode: an ID to identify each county
- county: the name of each county
- population: total population
- hispanic: percent of adults that are hispanic
- minority: percent of adults that are nonwhite
- female: percent of adults that are female
- unemployed: unemployment rate, as a percent
- income: median income
- nodegree: percent of adults who have not completed high school
- bachelor: percent of adults with a bachelor’s degree
- inactive: percent of adults who do not exercise in their leisure time
- obesity: percent of adults with BMI > 30
- density: population density, persons per square mile of land
- cancer: prevalence of cancer per 100,000 individuals
- voter_turnout: percentage of voting age population that voted
- voter_gap: percentage point gap in 2020 presidential voting: trump-briden
</div>

In [102]:
def merge_data(df, filepath):
    
    # YOUR CODE HERE
    election_df = pd.read_csv(filepath)
    election_df = election_df.drop(['fipscode', 'population'], axis = 1)

    merged = pd.merge(df, election_df,
                      left_on = ['state','county'], right_on = ['state','county'])

    # END OF YOUR CODE HERE
    return merged

Run the cell below (no changes necessary) to execute your code above

In [103]:
merged = merge_data(covid_updated, 'election2020_by_county.csv')

In [104]:
merged.head()

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
0,Nebraska,Hayes County,922,61,6616.052061,0,0.0,0.0,5.9,6.2,...,11.8,19.0,28.3,33.2,6.2,361.1,28.129206,86.1,92.5,6.4
1,Nebraska,Harlan County,3380,219,6479.289941,0,0.0,0.0,3.7,5.8,...,9.3,14.5,26.7,32.1,16.8,204.0,28.086535,69.2,83.8,14.6
2,Utah,Wayne County,2711,186,6860.936924,0,0.0,0.0,5.4,8.2,...,5.7,26.5,21.0,22.8,56.9,153.8,15.911486,52.9,75.6,22.7
3,Washington,San Juan County,17582,277,1575.474918,0,0.0,0.0,6.0,11.0,...,4.9,45.2,15.1,20.7,90.7,222.4,4.474074,-51.3,23.2,74.5
4,Nebraska,Keya Paha County,806,54,6699.751861,0,0.0,0.0,1.1,2.2,...,8.0,15.8,30.2,29.6,7.9,256.3,8.000000,80.7,90.0,9.3


In [50]:
merged.shape

(3012, 23)

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2.1 [1 pt]: Lost Rows</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [105]:
# YOUR CODE HERE
print(str(covid_updated.shape[0] - merged.shape[0]) + " rows were lost in the merge")


# END OF YOUR CODE HERE

69 rows were lost in the merge


<div class='exercise'><b>Exercise 4.2.2 [2 pts]: Lost Counties</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [106]:
# YOUR CODE HERE

# Since some counties in different states have the same name, we need both
# state and county name to uniquely id them. Concatenate and compare
covid_updated_countieslist = covid_updated['county'] + ", " + covid_updated['state']
merged_countieslist = merged['county'] + ", " + merged['state']

# find the set difference
set(covid_updated_countieslist) - set(merged_countieslist)

# END OF YOUR CODE HERE

{'Acadia Parish, Louisiana',
 'Aleutians West Census Area, Alaska',
 'Allen Parish, Louisiana',
 'Ascension Parish, Louisiana',
 'Assumption Parish, Louisiana',
 'Avoyelles Parish, Louisiana',
 'Beauregard Parish, Louisiana',
 'Bienville Parish, Louisiana',
 'Bossier Parish, Louisiana',
 'Caddo Parish, Louisiana',
 'Calcasieu Parish, Louisiana',
 'Caldwell Parish, Louisiana',
 'Cameron Parish, Louisiana',
 'Catahoula Parish, Louisiana',
 'Claiborne Parish, Louisiana',
 'Concordia Parish, Louisiana',
 'De Soto Parish, Louisiana',
 'Doña Ana County, New Mexico',
 'East Baton Rouge Parish, Louisiana',
 'East Carroll Parish, Louisiana',
 'East Feliciana Parish, Louisiana',
 'Evangeline Parish, Louisiana',
 'Franklin Parish, Louisiana',
 'Grant Parish, Louisiana',
 'Iberia Parish, Louisiana',
 'Iberville Parish, Louisiana',
 'Jackson Parish, Louisiana',
 'Jefferson Davis Parish, Louisiana',
 'Jefferson Parish, Louisiana',
 'Kalawao County, Hawaii',
 'Kusilvak Census Area, Alaska',
 'Lafayet

<div class='exercise'><b>Exercise 4.2.3 [2 pts]: Suggested Fixes</b>
   
If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

<div style='background-color:#F6FEFA;padding:15px'>

First, we should perform a "left" join rather than an "inner" join, as that will preserve the size of the larger dataframe (assuming the larger dataframe is on the left). We could populate cells with missing values as NA if the data are truly missing. If the data are missing because the join failed on state and county names, then we would need to fix the names. The best way to do this would not be to clean the names, but rather to use a unique identifier which is a string of numerals, which are easier to maintain than text (not case sensitive, for instance). We should use fips codes in this case. Finally, if we really wanted to get rid of NAs we could impute values using the values of neighboring counties, but I don't think that would lead to very reliable results.

</div>


This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [107]:
# YOUR CODE HERE
merged = merged.loc[merged['deaths'] != 0]
merged
# END OF YOUR CODE HERE

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
32,Nebraska,Saline County,14224,2044,14370.078740,2,14.060742,0.000978,24.3,29.3,...,17.1,14.3,31.0,33.0,1.5,309.1,40.293874,28.5,62.9,34.4
33,Colorado,Lake County,8127,938,11541.774332,1,12.304663,0.001066,33.9,35.7,...,7.4,30.3,15.1,17.5,19.4,112.4,29.853937,-20.2,37.9,58.1
34,Minnesota,Dodge County,20934,2369,11316.518582,3,14.330754,0.001266,4.9,7.5,...,6.5,24.1,18.3,24.9,709.0,140.3,17.427640,30.5,64.0,33.5
35,Colorado,Pitkin County,17767,2918,16423.706872,4,22.513649,0.001371,9.8,14.3,...,5.2,56.4,8.9,14.9,17.7,70.8,12.694664,-52.1,23.2,75.3
36,West Virginia,Calhoun County,7109,725,10198.340132,1,14.066676,0.001379,1.3,2.7,...,24.2,8.9,30.2,32.7,27.3,304.8,50.167729,60.5,79.6,19.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,Georgia,Dodge County,20605,1428,6930.356710,114,553.263771,0.079832,3.4,34.7,...,20.4,14.4,28.8,28.5,44.0,212.3,51.126453,45.5,72.4,26.9
3007,Georgia,Wilcox County,8635,610,7064.273306,50,579.038796,0.081967,4.3,41.5,...,23.9,8.7,27.7,31.5,24.5,248.4,53.295374,46.9,73.2,26.3
3008,Nebraska,Grant County,623,41,6581.059390,4,642.054575,0.097561,1.9,3.8,...,4.9,18.6,30.6,28.3,4.5,305.1,23.574144,88.3,93.3,5.0
3009,Georgia,Glascock County,2971,176,5923.931336,19,639.515315,0.107955,1.6,12.6,...,17.5,11.4,24.8,28.6,21.4,264.6,32.529082,79.7,89.6,9.9


Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [108]:
merged.describe()

,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,2.979000e+03,2.979000e+03,2979.000000,2979.000000,2979.000000,2979.000000,2978.000000,2978.000000,2978.000000,2979.000000,...,2979.000000,2979.000000,2979.000000,2979.000000,2979.000000,2938.000000,2947.000000,2946.000000,2946.000000,2946.000000
mean,1.062667e+05,1.306454e+04,12718.303864,208.533736,228.538832,0.018296,9.269913,22.521760,49.920151,5.488587,...,14.979859,19.993622,25.969151,30.993689,230.500705,228.605242,35.514568,33.039477,65.658282,32.618805
std,3.406025e+05,4.436774e+04,3685.969573,781.506718,119.066874,0.009811,13.934891,19.803921,2.357593,1.955033,...,6.746965,8.729129,5.157168,4.452694,1712.483148,55.843412,13.829715,30.882108,15.504336,15.387119
min,4.040000e+02,1.400000e+01,2240.000000,1.000000,8.299559,0.000978,0.000000,0.200000,19.166215,1.800000,...,1.900000,4.400000,8.100000,11.800000,0.100000,46.200000,-168.323353,-90.000000,4.000000,3.100000
25%,1.121350e+04,1.387000e+03,10429.252076,24.000000,143.403732,0.012181,2.000000,6.900000,49.465408,4.100000,...,9.900000,14.000000,22.700000,28.400000,17.300000,193.500000,27.701651,15.450000,56.825000,20.800000
50%,2.627100e+04,3.320000e+03,12670.670293,57.000000,212.222061,0.016479,4.000000,15.200000,50.384479,5.300000,...,13.500000,17.900000,25.800000,31.200000,45.000000,230.350000,35.049804,39.300000,68.800000,29.500000
75%,6.874050e+04,8.663000e+03,14875.162049,137.500000,293.301885,0.022418,9.500000,33.700000,51.069106,6.500000,...,19.200000,23.600000,29.400000,33.800000,110.350000,265.075000,42.476078,56.875000,77.575000,41.500000
max,1.003911e+07,1.371196e+06,72727.272727,25693.000000,865.800866,0.142857,99.200000,99.400000,56.633907,24.000000,...,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,100.000000,93.100000,96.200000,94.000000


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [109]:
def partition_df(df, column_name, minv, maxv):
    # YOUR CODE HERE
    partitioned = df.loc[(df[column_name] > minv) & (df[column_name] < maxv)]
    return partitioned
    # END OF YOUR CODE HERE

In [110]:
partition_df(merged, 'obesity', 30, 45)

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
32,Nebraska,Saline County,14224,2044,14370.078740,2,14.060742,0.000978,24.3,29.3,...,17.1,14.3,31.0,33.0,1.5,309.1,40.293874,28.5,62.9,34.4
36,West Virginia,Calhoun County,7109,725,10198.340132,1,14.066676,0.001379,1.3,2.7,...,24.2,8.9,30.2,32.7,27.3,304.8,50.167729,60.5,79.6,19.1
37,Nebraska,Jefferson County,7046,669,9494.748794,1,14.192450,0.001495,3.6,5.8,...,10.5,13.2,28.0,36.2,11.0,230.7,33.996448,43.1,70.4,27.3
38,Kentucky,Elliott County,7517,1208,16070.240788,2,26.606359,0.001656,0.3,3.9,...,26.4,6.4,34.0,33.7,40.2,263.5,51.030085,51.2,75.0,23.8
39,Minnesota,Wabasha County,21627,2378,10995.514866,4,18.495399,0.001682,2.9,5.1,...,7.3,20.7,21.5,32.2,6.2,317.1,21.197353,26.4,62.3,35.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,Georgia,Telfair County,15860,916,5775.535939,62,390.920555,0.067686,11.6,50.9,...,25.7,10.8,29.0,30.9,37.7,187.9,62.239651,30.9,65.2,34.3
3001,Georgia,Terrell County,8531,752,8814.910327,53,621.263627,0.070479,2.7,64.5,...,31.6,10.1,27.8,31.6,27.8,258.2,34.809566,-8.4,45.4,53.8
3003,Georgia,Hancock County,8457,981,11599.858106,71,839.541208,0.072375,1.8,75.9,...,27.6,10.6,27.8,31.2,20.0,232.5,42.757009,-43.9,27.8,71.7
3004,Georgia,Twiggs County,8120,701,8633.004926,51,628.078818,0.072753,0.4,45.1,...,31.2,9.6,29.6,32.8,25.2,234.8,33.253229,7.3,53.3,46.0


<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [111]:
# YOUR CODE HERE
merged.describe()
# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,2.979000e+03,2.979000e+03,2979.000000,2979.000000,2979.000000,2979.000000,2978.000000,2978.000000,2978.000000,2979.000000,...,2979.000000,2979.000000,2979.000000,2979.000000,2979.000000,2938.000000,2947.000000,2946.000000,2946.000000,2946.000000
mean,1.062667e+05,1.306454e+04,12718.303864,208.533736,228.538832,0.018296,9.269913,22.521760,49.920151,5.488587,...,14.979859,19.993622,25.969151,30.993689,230.500705,228.605242,35.514568,33.039477,65.658282,32.618805
std,3.406025e+05,4.436774e+04,3685.969573,781.506718,119.066874,0.009811,13.934891,19.803921,2.357593,1.955033,...,6.746965,8.729129,5.157168,4.452694,1712.483148,55.843412,13.829715,30.882108,15.504336,15.387119
min,4.040000e+02,1.400000e+01,2240.000000,1.000000,8.299559,0.000978,0.000000,0.200000,19.166215,1.800000,...,1.900000,4.400000,8.100000,11.800000,0.100000,46.200000,-168.323353,-90.000000,4.000000,3.100000
25%,1.121350e+04,1.387000e+03,10429.252076,24.000000,143.403732,0.012181,2.000000,6.900000,49.465408,4.100000,...,9.900000,14.000000,22.700000,28.400000,17.300000,193.500000,27.701651,15.450000,56.825000,20.800000
50%,2.627100e+04,3.320000e+03,12670.670293,57.000000,212.222061,0.016479,4.000000,15.200000,50.384479,5.300000,...,13.500000,17.900000,25.800000,31.200000,45.000000,230.350000,35.049804,39.300000,68.800000,29.500000
75%,6.874050e+04,8.663000e+03,14875.162049,137.500000,293.301885,0.022418,9.500000,33.700000,51.069106,6.500000,...,19.200000,23.600000,29.400000,33.800000,110.350000,265.075000,42.476078,56.875000,77.575000,41.500000
max,1.003911e+07,1.371196e+06,72727.272727,25693.000000,865.800866,0.142857,99.200000,99.400000,56.633907,24.000000,...,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,100.000000,93.100000,96.200000,94.000000


In [112]:
partition_df(merged, 'trump', 0, 56.825).describe()

,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,7.370000e+02,7.370000e+02,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,737.000000,...,737.000000,737.000000,737.000000,737.000000,737.000000,731.000000,737.000000,737.000000,737.000000,737.000000
mean,2.999259e+05,3.571548e+04,11670.482174,566.693351,199.624832,0.016851,13.700678,38.312483,50.399151,5.927680,...,14.143555,26.888738,22.878697,29.451696,404.214383,216.728044,35.959351,-10.485075,43.819132,54.304206
std,6.342038e+05,8.358019e+04,3851.657467,1489.287555,125.321563,0.009066,18.867704,23.722498,2.169557,2.336192,...,7.864929,11.611929,5.651173,6.076583,1748.003627,59.201060,20.305757,20.758919,10.353074,10.425191
min,7.280000e+02,8.600000e+01,2484.334583,1.000000,8.299559,0.001066,0.000000,2.500000,32.813627,2.200000,...,1.900000,6.000000,8.100000,11.800000,0.300000,46.200000,-168.323353,-90.000000,4.000000,40.700000
25%,2.499400e+04,2.674000e+03,9239.508463,45.000000,106.760905,0.010726,3.000000,17.400000,49.898763,4.300000,...,8.400000,17.900000,18.700000,25.500000,23.000000,178.500000,24.879747,-23.100000,37.500000,45.900000
50%,9.252500e+04,9.456000e+03,11633.827762,129.000000,169.546677,0.015170,6.200000,35.700000,50.704225,5.400000,...,11.800000,26.000000,22.500000,29.300000,63.000000,216.600000,33.241396,-6.200000,45.900000,52.000000
75%,3.183160e+05,3.477500e+04,13969.206159,518.000000,262.346858,0.020744,15.000000,56.800000,51.500966,6.900000,...,18.600000,34.000000,26.300000,33.200000,243.900000,252.400000,44.071825,6.600000,52.400000,60.500000
max,1.003911e+07,1.371196e+06,40909.507656,25693.000000,839.541208,0.072753,99.200000,99.400000,55.797167,24.000000,...,53.300000,72.000000,38.200000,47.600000,32903.300000,458.300000,99.552895,15.900000,56.800000,94.000000


In [113]:
print("Mean cases per 100k by minority quartiles")
print("First quartile:")
print(partition_df(merged, 'minority', 0, 6.9)['cases_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'minority', 6.9, 15.2)['cases_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'minority', 15.2, 33.7)['cases_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'minority', 33.7, 100)['cases_per_100k'].mean())
print("\n")

print("Mean cases per 100k by trump quartiles")
print("First quartile:")
print(partition_df(merged, 'trump', 0, 56.825)['cases_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'trump', 56.825, 68.8)['cases_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'trump', 66.8, 77.575)['cases_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'trump', 77.575, 100)['cases_per_100k'].mean())
print("\n")

print("Mean deaths per 100k by minority quartiles")
print("First quartile:")
print(partition_df(merged, 'minority', 0, 6.9)['deaths_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'minority', 6.9, 15.2)['deaths_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'minority', 15.2, 33.7)['deaths_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'minority', 33.7, 100)['deaths_per_100k'].mean())
print("\n")

print("Mean deaths per 100k by trump quartiles")
print("First quartile:")
print(partition_df(merged, 'trump', 0, 56.825)['deaths_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'trump', 56.825, 68.8)['deaths_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'trump', 66.8, 77.575)['deaths_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'trump', 77.575, 100)['deaths_per_100k'].mean())
print("\n")

print("Mean cases per 100k by biden quartiles")
print("First quartile:")
print(partition_df(merged, 'biden', 0, 20.8)['cases_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'biden', 20.8, 29.5)['cases_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'biden', 29.5, 41.5)['cases_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'biden', 41.5, 100)['cases_per_100k'].mean())
print("\n")

print("Mean deaths per 100k by biden quartiles")
print("First quartile:")
print(partition_df(merged, 'biden', 0, 20.8)['deaths_per_100k'].mean())
print("Second quartile:")
print(partition_df(merged, 'biden', 20.8, 29.5)['deaths_per_100k'].mean())
print("Third quartile:")
print(partition_df(merged, 'biden', 29.5, 41.5)['deaths_per_100k'].mean())
print("Fourth quartile:")
print(partition_df(merged, 'biden', 41.5, 100)['deaths_per_100k'].mean())

# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

Mean cases per 100k by minority quartiles
First quartile:
12127.819625593467
Second quartile:
12563.894296664117
Third quartile:
12581.44753985462
Fourth quartile:
13597.05228555903


Mean cases per 100k by trump quartiles
First quartile:
11670.482174088746
Second quartile:
12474.431148446498
Third quartile:
13294.444933828107
Fourth quartile:
13423.834664229142


Mean deaths per 100k by minority quartiles
First quartile:
223.30402151766154
Second quartile:
204.20720269002044
Third quartile:
210.62770298008098
Fourth quartile:
276.5963333264724


Mean deaths per 100k by trump quartiles
First quartile:
199.62483211911442
Second quartile:
212.9376082465337
Third quartile:
238.9126587229542
Fourth quartile:
263.4544128785315


Mean cases per 100k by biden quartiles
First quartile:
13342.03793251125
Second quartile:
13480.050240967877
Third quartile:
12419.222386472873
Fourth quartile:
11682.712405442226


Mean deaths per 100k by biden quartiles
First quartile:
259.7921344572366
Second qua

<div style='background-color:#F6FEFA;padding:15px'>

**Here we can notice that the latter the quartiles per the biden variable (i.e. the more people voted for Biden), the lower the mean deaths and cases. Hinting at how on average states and counties that are more Democrats might have taken the pandemic more seriously (e.g. masks usage, social distancing), and hence had a lower amount of deaths and cases on average. The opposite is the case for the variable trump. The latter the quartiles, i.e. the more people voted for Trump, the higher the average of deaths and cases.
    Additionally, the higher the minority presence is, the higher the average of cases and deaths are present as well.**
</div>


`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [114]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

,state,county,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,...,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden,income group
32,Nebraska,Saline County,14224,2044,14370.078740,2,14.060742,0.000978,24.3,29.3,...,14.3,31.0,33.0,1.5,309.1,40.293874,28.5,62.9,34.4,income-group-3
33,Colorado,Lake County,8127,938,11541.774332,1,12.304663,0.001066,33.9,35.7,...,30.3,15.1,17.5,19.4,112.4,29.853937,-20.2,37.9,58.1,income-group-3
34,Minnesota,Dodge County,20934,2369,11316.518582,3,14.330754,0.001266,4.9,7.5,...,24.1,18.3,24.9,709.0,140.3,17.427640,30.5,64.0,33.5,income-group-4
35,Colorado,Pitkin County,17767,2918,16423.706872,4,22.513649,0.001371,9.8,14.3,...,56.4,8.9,14.9,17.7,70.8,12.694664,-52.1,23.2,75.3,income-group-4
36,West Virginia,Calhoun County,7109,725,10198.340132,1,14.066676,0.001379,1.3,2.7,...,8.9,30.2,32.7,27.3,304.8,50.167729,60.5,79.6,19.1,income-group-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,Georgia,Dodge County,20605,1428,6930.356710,114,553.263771,0.079832,3.4,34.7,...,14.4,28.8,28.5,44.0,212.3,51.126453,45.5,72.4,26.9,income-group-1
3007,Georgia,Wilcox County,8635,610,7064.273306,50,579.038796,0.081967,4.3,41.5,...,8.7,27.7,31.5,24.5,248.4,53.295374,46.9,73.2,26.3,income-group-1
3008,Nebraska,Grant County,623,41,6581.059390,4,642.054575,0.097561,1.9,3.8,...,18.6,30.6,28.3,4.5,305.1,23.574144,88.3,93.3,5.0,income-group-3
3009,Georgia,Glascock County,2971,176,5923.931336,19,639.515315,0.107955,1.6,12.6,...,11.4,24.8,28.6,21.4,264.6,32.529082,79.7,89.6,9.9,income-group-2


<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [115]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby('income group').agg('mean')



# END OF YOUR CODE HERE

,population,cases,cases_per_100k,deaths,deaths_per_100k,case_fatality_rate,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
income group,,,,,,,,,,,,,,,,,,,,,
income-group-1,27822.511450,3967.729771,13938.683609,85.535878,297.571746,0.022672,9.138779,32.366718,50.002613,7.272366,...,21.991145,13.595420,30.101985,33.647328,87.845954,238.566258,41.844069,33.203988,65.926074,32.722086
income-group-2,64545.897909,8688.885609,12794.250094,144.731857,237.898358,0.019012,8.869704,20.386946,49.813455,5.718696,...,15.617712,17.565068,26.996187,31.489914,131.076630,235.296981,37.075902,40.729543,69.545117,28.815575
income-group-3,98807.927476,12957.001395,12523.820423,193.654114,210.900509,0.016994,8.860669,18.001534,49.863219,4.852301,...,12.760530,20.438773,25.085914,30.619386,341.441562,227.691926,34.282463,35.691831,66.912254,31.220423
income-group-4,220432.837531,25146.321159,11809.426796,388.764484,177.935336,0.015128,10.156927,20.665365,50.012650,4.356045,...,10.546977,27.356423,22.305793,28.634509,349.803023,214.376306,29.680387,22.443742,60.226839,37.783097


<div class='exercise'><b>Wrapping Up</b>
</div>

<div class='exercise'><b>Exercise 4.7.1 [1 pt]: Conclusions</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

Here from 4.6 we can notice that the higher the income, the lower the cases per 100k and deaths per 100k, as well as the case fatality rate. 
    
    
By using this kind of visualization we could ideentify possible patterns, e.g. population with greater income have better healthcare and jobs (i.e. the opportunity to work from home and practice social distancing).
    
On the other hand, low income families most likely work in jobs in person, putting their families and themselves at risk, possibly increasing the COVID spread and contamination rate. 


</div>


<div class='exercise'><b>Exercise 4.7.2 [1 pt]: Possible Weaknesses</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

Comparing the later way of representing data with the intial ones from problem 4, we can observe that in the effort of making the representation readable, we do not include all the informations provided by the data. Thus we are in danger of missing an important correlation of data that is not related with the income of the subjects.
</div>


## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.